In [138]:
import requests
from bs4 import BeautifulSoup
import typing

In [86]:
from enum import Enum
class Constants(Enum):
    CMU_SYLLABUS_REGISTRY = 'https://canvas.cmu.edu/courses/sis_course_id:syllabus-registry' 
    COOKIE = {
        'canvas_session': 'L88s0oZPvegNtlI0txEL4Q+QRmf1xZxzWYv3AKbVMBiZ8N9ZWcbIzZnuQWoBUcEELKL1zoysNBkag1Ba-UCLlnSzvOsEwdeWpPO37Y1gl37oT0FMifmjZRM6dBFbthjh9XlrFclHPYWbSDMuC8PRPQNXM2CLmg6CISd_STyDL2C6rn08tXKbeLCKh8gg-lIIf5HWpvIBAk24XQtioG4r0MGYP2gNB5nO0247PD_mtzufNjuFlbUOMaMtR-k3N-70l0aZ1z1x4aynW26enj4Y8y0-1pfoVuDEE3XaAQAdhPbYPdGIAzQiClBwCx19hCjSQ4seWs091TR9wv6BDbAQZBonI1KL4Lm_zt7vkCBNehWQhfcst4e--wsMINxSxbwNh47Z2FHu3xldcIu3TVjLa56jNilLwni9ftqMRA1OhfNCdHi9XwJ5fvfWukCCdVRMjjyuJT_uBU6IDQ61J25iQqgjNd1ud58eJw1IheM7untH8_44AFjIsCrmN24LFOwAEIvqzENFHLsw4ANER2O0LKUP_vPl2403y3uuuAQo2j5L-L8-zPfnOsAYRBYsxXPGCHnLZHChr1yHAxOnqAmAb8xbt2T-fKyBxDWjEpw1BgL_GokbZB16Zt3BF1Cadg0sc4VUemaNXUPtzQVmAACUuxaLWGRCkb1bgPLDnf0bMMsNkgVxEkL-cCgrNnjArjmRkROucKf6wP6DMSi2HdW1esYPffOZoIyHEUqi4aTaawHysVVmymEkgqr1cbJ7KRx83kYQXif6zuBJEWd9kaemA6PiTzXa9mBU_LPUlldnVKjx50hIS0fSXB1tMWsh9OJOZIziJXs6wzknclEdejDRZndyld-aTLOLbElVKIT4l2zDg.uPhGgHIPBeqXNvHKk8O0Hyo10gk.Z1a-QQ'
    }

In [146]:
def get_and_unwrap(*args, **kwargs) -> BeautifulSoup:
    response = requests.get(*args, **kwargs)
    assert response.status_code == 200
    return BeautifulSoup(response.text, 'html.parser')

In [141]:
soup = get_and_unwrap(Constants.CMU_SYLLABUS_REGISTRY.value)

In [142]:
semester = soup.find(attrs={"aria-label": "Fall 2024 (F24)"})
semesters = soup.select('div[aria-label]')
# for semester in semesters:
#     print(semester.get('aria-label'))

In [143]:
departments = semester.select('div.content > ul.context_module_items > li[id^="context_module_item_"] > div.ig-row > div.ig-info > div.module-item-title > span.item_name')

In [144]:
def select_unique(html, *args, **kwargs):
    result = html.select(*args, **kwargs)
    assert len(result) == 1, 'Selection is not unique!'
    return result[0]

In [152]:
class Course:
    def __init__(self, html):
        self.html = html
        self.a = select_unique(self.html, 'a.ig-title')
    @property 
    def name(self):
        return self.a.get('title')
    @property
    def href(self):
        return self.a.get('href')
        
class Department:
    def __init__(self, html):
        self.html = html
        self.a = select_unique(self.html, 'a.external_url_link')
    @property 
    def name(self):
        return self.a.get('title')
    @property
    def href(self):
        return self.a.get('href')
    @staticmethod
    def get_category(html, cat: str):
        category = select_unique(html, f'div[aria-label="{cat}"]')
        courses = category.select('div.content > ul.context_module_items > li[id^="context_module_item_"] > div.ig-row > div.ig-info > div.module-item-title > span.item_name')
        return courses
    def get(self):
        html = get_and_unwrap(self.href, cookies=Constants.COOKIE.value)
        courses = self.get_category(html, "Available Syllabi")
        self.courses = [Course(c) for c in courses]

class Semester:
    def __init__(self, html):
        self.html = html
        departments = semester.select('div.content > ul.context_module_items > li[id^="context_module_item_"] > div.ig-row > div.ig-info > div.module-item-title > span.item_name')
        self.departments = [Department(d) for d in departments]
    @property 
    def name(self):
        return self.html.get('aria-label')

class SyllabusRegistry:
    def __init__(self):
        html = get_and_unwrap(Constants.CMU_SYLLABUS_REGISTRY.value)
        self.html = html
        semesters = html.select('div[aria-label^="Fall"], div[aria-label^="String"], div[aria-label^="Summer"]')
        self.semesters = [Semester(s) for s in semesters]

In [41]:
sr = SyllabusRegistry()

In [153]:
s = Semester(sr.semesters[0].html)

In [157]:
s.departments[0].get()

In [158]:
s.departments[0].courses[0].name

'48095-A: Spatial Concepts for Non-Architecture Majors'